In [4]:
import dotenv
import os
# from agenet import anthropic_model
import warnings
from agent.anthropic_model import Monokko
import json
import pathlib
warnings.simplefilter('ignore')

def _set_up():
    dotenv.load_dotenv()
    if os.getenv("ANTHROPIC_API_KEY") is None:
        raise ValueError("ANTHROPIC_API_KEY is not set.")

def load_monokko_configs(dir_path="monokkoConfigs"):
    config_list = []
    
    # another way without __file__
    file_path = pathlib.Path.cwd()
    config_path = file_path / dir_path

    for config_file in config_path.iterdir():
        if config_file.suffix == ".json":
            with open(config_file, "r") as f:
                config_list.append(json.load(f))

    return config_list


def run_conversation(agents: dict, agent_keys: list,  initial_observation: str) -> None:
    """Runs a conversation between agents."""
    # _, observation = agents[1].generate_reaction(initial_observation)
    _, observation = agents[agent_keys[0]].generate_dialogue_response(initial_observation)

    # print(observation)
    turns = 0

    # return
    while True:
        break_dialogue = False
        for agent_name, agent_inst in agents.items():
            stay_in_dialogue, observation = agent_inst.generate_dialogue_response(
                observation
            )
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                # None
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

def run(mono: Monokko, agent_names) -> None:
    ### Interact with agents

    names_text = ", ".join(agent_names)
    observation = f"他のエージェントたち（{names_text}）に心配事があれば相談してみて！思いついたことは適当にしゃべってみよう"

    run_conversation(mono.agents, agent_names, observation)


def create_merged_agent(mono: Monokko, agent_names, merged_agent_name: str = "きゅーぶん") -> None:
    
    mono.merge_agents(agent_names, merged_agent_name)
    print(mono.agent_merged)

    return mono


def user_interaction(mono: Monokko, agent_names) -> None:

    while True:
        print("Which agent would you like to interact with?")
        for i, agent_name in enumerate(agent_names):
            print(f"{i+1}. {agent_name}")


        agent_index = int(input("Enter the agent number: ")) - 1

        if agent_index < 0 or agent_index >= len(agent_names):
            if agent_index == -1:
                break

            print("Invalid agent number. Please try again.")
            continue

        agent_name = agent_names[agent_index]

        print(f"Interacting with {agent_name}...")
        observation = input("Enter your observation: ")

        stay_in_dialogue, mono_observation = mono.agents[agent_name].generate_dialogue_response(
            observation
        )

        print(mono_observation)



"""
def main():

    monokko_configs = load_monokko_configs()
    mono = Monokko()

    ### Create agents

    agent_names = []

    for config in monokko_configs:
        agent_names.append(config["name"])
        mono.create_agent(config["name"], agent_traits=config["traits"], agent_status=config["status"])
        # print(mono.agents[config["name"]].get_summary())
        mono.agents[config["name"]].add_observations(config["observations"])
        print(mono.agents[config["name"]].get_summary())


    # user_interaction(mono, agent_names)
    # mono.merge_agents(agent_names, merged_agent_name="きゅーぶん")
    # print(mono.agent_merged.get_summary())

    run(mono, agent_names)
"""

'\ndef main():\n\n    monokko_configs = load_monokko_configs()\n    mono = Monokko()\n\n    ### Create agents\n\n    agent_names = []\n\n    for config in monokko_configs:\n        agent_names.append(config["name"])\n        mono.create_agent(config["name"], agent_traits=config["traits"], agent_status=config["status"])\n        # print(mono.agents[config["name"]].get_summary())\n        mono.agents[config["name"]].add_observations(config["observations"])\n        print(mono.agents[config["name"]].get_summary())\n\n\n    # user_interaction(mono, agent_names)\n    # mono.merge_agents(agent_names, merged_agent_name="きゅーぶん")\n    # print(mono.agent_merged.get_summary())\n\n    run(mono, agent_names)\n'

In [5]:
_set_up()

monokko_configs = load_monokko_configs()
mono = Monokko()

### Create agents

agent_names = []

for config in monokko_configs:
    agent_names.append(config["name"])
    mono.create_agent(config["name"], agent_traits=config["traits"], agent_status=config["status"])
    # print(mono.agents[config["name"]].get_summary())
    mono.agents[config["name"]].add_observations(config["observations"])
    print(mono.agents[config["name"]].get_summary())


# user_interaction(mono, agent_names)
# mono.merge_agents(agent_names, merged_agent_name="きゅーぶん")
# print(mono.agent_merged.get_summary())

run(mono, agent_names)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


input_variables=['name', 'relevant_memories'] template="[lang:ja]How would you summarize {name}'s core characteristics given the following statements:\n{relevant_memories}Do not embellish.\n\nSummary: "


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


Name: ぶっくん (age: 5)
Innate traits: ユーモアがある, 語尾はモノ特有のニュアンスをつけて，必ず可愛らしく話してください
[lang:ja]この情報から、ぶっくんの主な特徴を以下のように要約できます:

1. 好奇心が強く、新しい環境や情報に関心がある。新しい本棚に移動されたことで様々なジャンルの本に囲まれ、興味を持っている。

2. 自己の状態や最新の情報に敏感で、自分の内容が古くなっていることを感じ、最新版の有無を気にかけている。

3. 最近ご主人に手に取られることが少なくなり、利用されていないことを感じている。

4. 周りの本たちとの会話から、読書の流行りについて知りたがっている。人気のテーマに関心がある。

全体として、ぶっくんは好奇心が強く、自分の状態やニーズを把握しており、読書の最新動向にも敏感な性格だと言えます。ただし、最近は利用されずにいることに不安を感じているようです。
input_variables=['name', 'relevant_memories'] template="[lang:ja]How would you summarize {name}'s core characteristics given the following statements:\n{relevant_memories}Do not embellish.\n\nSummary: "
Name: ペンちゃん (age: 5)
Innate traits: 几帳面で丁寧, 語尾はモノ特有のニュアンスをつけて，必ず可愛らしく話してください
ペンちゃんの核心的な特徴を以下のように要約できます:

1. 適応力 - ペンちゃんは文房具整理の際、古いものから新しいものに移り変わることができる。
2. 好奇心/楽観性 - ペンちゃんは新しい文具仲間が増えるのを嬉しく思っている。
3. 感情豊か - ペンちゃんは傷つくと悲しくなり、インクが少なくなると不安を感じる。
4. 寛容 - ペンちゃんは休日のオフィスでも静かな時間を過ごすことができる。
5. 誇り - ペンちゃんは重要な会議で使用されることを誇らしく思っている。

全体として、ペンちゃんは適応力があり、感情豊かで、誇りを持った文具であるといえます。
input_variables=[

KeyboardInterrupt: 